In [1]:
import time
import csv
import os
from pathlib import Path
import sys
sys.path.insert(0, '../')
import pycor 
from pycor import utils, korlang

stopwatch = utils.StopWatch()


model_size = 7000
outputpath = "../../output/" + str(model_size) + "/"
model_path = outputpath + "model/"

pycor.loadmodel(model_path)



Init Trainer
Init DocResolver
Loading Model from ../../output/7000/model/
Load  ../../output/7000/model//heads.csv   소요시간: 1.399
Load  ../../output/7000/model//tails.csv   소요시간: 0.011


In [2]:
def listfiles(path):
    result_arr = []
    filenames = os.listdir(path)
    for filename in filenames:
        full_filename = os.path.join(path, filename)
        result_arr.append(full_filename)
    return result_arr
    
files = []
files.extend( listfiles('../../data/wiki2') )
files.extend( listfiles('../../data/wiki') )
files.extend( listfiles('../samples/news') )
files.extend( listfiles('../../data/news') )
files.extend( listfiles('../../data/blotter') )
files.extend( listfiles('../../data/zdnet') )
files.extend( listfiles('../../data/ciobiz') )
files.extend( listfiles('../../data/NP') )

print (len(files), "files")

3959 files


In [3]:

yeTags = set(['EFN'])

def gettail(word):
    if word.bestpair:
#         return word.bestpair.tags
        return word.bestpair.tail.text
    return 'X'

def gettag(word):
    if word.bestpair:
        return word.bestpair.tags
    return 'X'

def gethead(word):
    if word.bestpair:
        return word.bestpair.head.text
    return 'X'


yheadsMap = {}
ytailsMap = {}
ytagsMap = {}

def handleHaed(index, word, words):
    yset = yheadsMap.get(word.bestpair.head)
    if yset is None:
        yset = []
        yheadsMap[word.bestpair.head] = yset

    if index > 1:
        yset.append( [gethead(words[index-1]),gethead(words[index-2])])
    else:
        yset.append( [gethead(words[index-1])] )
        
def handleTag(index, word, words):
    yset = ytagsMap.get(word.bestpair.head)
    if yset is None:
        yset = []
        ytagsMap[word.bestpair.head] = yset
    if index > 1:
        yset.append( [gettag(words[index-1]),gettag(words[index-2])])
    else:
        yset.append( [gettag(words[index-1])] )
                
def handleTail(index, word, words):
    yset = ytailsMap.get(word.bestpair.head)
    if yset is None:
        yset = []
        ytailsMap[word.bestpair.head] = yset
    if index > 1:
        yset.append( [gettail(words[index-1]),gettail(words[index-2])])
    else:
        yset.append( [gettail(words[index-1])] )
        
def analyzeYongEon(file):
    sentence_array, words_array = pycor.readfile(file)
    
    for words in words_array:
        for index, word in enumerate(words):
            if word.bestpair and len(yeTags & set(word.bestpair.tags))>0 and word.text.endswith('다'):
                handleHaed(index, word, words)
                handleTail(index, word, words)
                handleTag(index, word, words)


def writeFile(path, yMap):
    with open(path, 'w', encoding='utf-8') as file :
        writer = csv.writer(file)
        for head, tags in yMap.items():
            headText = head.text
            for tag in tags:
                row = [headText]
                row.extend(tag)
                writer.writerow(row)
        file.close()
                         
stopwatch.start()

docsize = 7000

for file in files[:docsize]:
    if file.endswith(".txt"):
        analyzeYongEon(file)

print("Loading ",docsize," Docs: " , stopwatch.secmilli() , "(", stopwatch.millisecstr(), "ms.)")

Loading  7000  Docs:  540s.186ms. ( 540,186 ms.)


In [22]:
# yheadsMap 
# ytailsMap  
# ytagsMap  

def fill(ylist, ymap):
    for head, tags in ymap.items():
        headText = head.text
        for tag in tags:
            row = [headText]
            row.extend(tag)
            ylist.append(row) 
        
tailsList = []
tagsList = []

fill(tailsList,ytailsMap)
fill(tagsList,ytagsMap)

josaYMap = {}
matrix = []

for index in range(len(tailsList)):
    tail = tailsList[index]
    tag = tagsList[index]
    
    text = tail[0]
    row = [text]
    row.extend(tail[1:])
    
    josaKey = ''
    for tags in tag[1:]:
        tagsStr = "+".join(tags)
        row.append(tagsStr)
        josaKey += "::"
        for t in tags:
            if t.startswith("J"):
                josaKey += "+" + t
    
    ymap = josaYMap.get(josaKey)
    if ymap is None:
        ymap = {}
        josaYMap[josaKey] = ymap
    
    count = ymap.get(text)
    if count is None:
        count = 0
    
    count += 1
    ymap[text] = count
    
    matrix.append(row)

    
def writeTailTagsFile(path, tailTagsList):
    with open(path, 'w', encoding='utf-8') as file :
        writer = csv.writer(file)
        for row in tailTagsList:
            writer.writerow(row)
                
        file.close()
        
outputpath = "../../output/" + str(docsize) + "/"

writeFile(outputpath+'/yongeon-heads.csv',yheadsMap)
# writeFile(outputpath+'/yongeon-tails.csv',ytailsMap)
# writeFile(outputpath+'/yongeon-tags.csv',ytagsMap)


writeTailTagsFile(outputpath+'/yongeon-tails-tags.csv',matrix)

print(len(matrix))

print(len(josaYMap))

190087
798


In [23]:
for tags, ymap in josaYMap.items():
    print(tags, len(ymap))
#     for y, count in ymap.items():
#         print("  ", y, count)
        

::+JKO::+JKG 1172
::+JKO::+JX-SO 856
::+JKO+JKP:: 66
::+JKO:: 2763
::::+JX-SO 1346
::+JKO::+JKG+JKB-TT|AS|BY 23
::+JKO::+JX 149
::+JKO::+JKS 466
:::: 3720
::+JKO::+JKB-WZ 385
::+JKO::+JX-SO+JKB-TT|AS|BY 18
::+JKO 145
::+JKO::+JKO 63
::+JKO::+JKB-TT|AS|BY 523
::+JKO+JKP::+JX-SO 9
::+JKO::+JX+JKP 40
::+JKO::+JC 183
::+JKO::+JKG+JC 49
::+JKB-TT|AS|BY::+JKO 603
::::+JKP 95
::+JKO::+JKB-FM 244
::::+JKG 285
::+JKB-TO::+JKO 250
::+JKB-TT|AS|BY:: 1825
::+JKB-TT|AS|BY::+JKG 535
::+JX-SO+JKQ+JKP:: 9
::+JKQ:: 227
::+JX-SO+JKQ:: 44
::+JKQ+JKP::+JKB-TO 4
::+JX-SO+JKQ+JKP::+JKG 3
::+JKQ::+JKG 30
::+JKB-TT|AS|BY::+JKP 47
::+JKP::+JX-SO 32
::+JKB-TT|AS|BY::+JX-SO 561
::+JKP:: 99
::+JX-SO+JKB-TT|AS|BY::+JX-SO 14
::+JKQ+JKP::+JX-SO 31
::+JKQ+JKP:: 113
::+JX::+JX-SO+JKB-TO 15
::+JX-SO+JKQ+JKP::+JX-SO 4
::+JX-SO+JKB-TT|AS|BY:: 53
::::+JKB-TO 1103
::::+JKO 1852
::::+JX-SO+JKB-TO 80
::+JKS:: 1879
::+JX-SO+JKQ::+JKG 5
::+JKQ::+JX-SO+JKB-TO 3
::+JKO::+JKV 220
::+JKQ::+JX-SO 27
::+JX-SO+JKQ+JKP::+JKB-TO 1
::+J